In [1]:
!pip install pandas
!pip install openpyxl
!pip install requests
!pip install tqdm


In [2]:
import pandas as pd
from main import create_new_weather_columns

In [3]:
df = pd.read_excel("target.xlsx", sheet_name="data cleaned")

In [4]:
states = df["state"].unique()
# print(states)
df

,start_date,year,month,nps,state
0,2024-10-01,2024,10,8,Алтайский край
1,2024-10-01,2024,10,10,Алтайский край
2,2024-10-01,2024,10,10,Алтайский край
3,2024-10-01,2024,10,10,Алтайский край
4,2024-10-01,2024,10,10,Алтайский край
...,...,...,...,...,...
71495,2024-04-01,2024,4,10,Ярославская область
71496,2024-04-01,2024,4,3,Ярославская область
71497,2024-04-01,2024,4,0,Ярославская область
71498,2024-04-01,2024,4,5,Ярославская область


In [5]:
# One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=['state'])
df_encoded.head(2)

,start_date,year,month,nps,state_Алтайский край,state_Амурская область,state_Архангельская область,state_Астраханская область,state_Брянская область,state_Владимирская область,...,state_Томская область,state_Тульская область,state_Тюменская область,state_Ульяновская область,state_Хабаровский край,state_Ханты-Мансийский автономный округ - Югра,state_Челябинская область,state_Чеченская Республика,state_Чувашская Республика - Чувашия,state_Ярославская область
0,2024-10-01,2024,10,8,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2024-10-01,2024,10,10,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
average_nps_by_region = df.groupby('state')['nps'].mean()

print("Средний NPS по регионам:")
print(average_nps_by_region)

Средний NPS по регионам:
state
Алтайский край                              7.181538
Амурская область                            6.962308
Архангельская область                       5.994615
Астраханская область                        6.715385
Брянская область                            5.810000
Владимирская область                        5.808462
Волгоградская область                       6.690769
Вологодская область                         6.269231
Воронежская область                         5.534615
Забайкальский край                          7.483077
Ивановская область                          7.150000
Иркутская область                           5.780769
Кабардино-Балкарская Республика             5.720769
Карачаево-Черкесская Республика             5.360000
Кемеровская область                         6.641538
Кировская область                           7.015385
Краснодарский край                          6.191538
Красноярский край                           6.262308
Ленинградская о

In [7]:
df_weather = create_new_weather_columns(df, cache_path='../cache/weather_cache.pkl')

Cache file '../cache/weather_cache.pkl' успешно загружен.


Processing rows: 100%|██████████| 71500/71500 [00:01<00:00, 65328.91it/s]


Cache saved to ../cache/weather_cache.pkl


In [8]:
df_weather

,start_date,year,month,nps,state,temp_max_avg,temp_min_avg,precipitation_sum,wind_speed_max_avg,days_clear,days_cloudy,days_rain_drizzle,days_snow
0,2024-10-01,2024,10,8,Алтайский край,10.905000,2.430000,1.651667,18.390000,2,24,27,7
1,2024-10-01,2024,10,10,Алтайский край,10.905000,2.430000,1.651667,18.390000,2,24,27,7
2,2024-10-01,2024,10,10,Алтайский край,10.905000,2.430000,1.651667,18.390000,2,24,27,7
3,2024-10-01,2024,10,10,Алтайский край,10.905000,2.430000,1.651667,18.390000,2,24,27,7
4,2024-10-01,2024,10,10,Алтайский край,10.905000,2.430000,1.651667,18.390000,2,24,27,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71495,2024-04-01,2024,4,10,Ярославская область,8.324194,-1.437097,0.977419,19.069355,0,36,20,6
71496,2024-04-01,2024,4,3,Ярославская область,8.324194,-1.437097,0.977419,19.069355,0,36,20,6
71497,2024-04-01,2024,4,0,Ярославская область,8.324194,-1.437097,0.977419,19.069355,0,36,20,6
71498,2024-04-01,2024,4,5,Ярославская область,8.324194,-1.437097,0.977419,19.069355,0,36,20,6
